### 참고) [word2vec](https://shuuki4.wordpress.com/2016/01/27/word2vec-%EA%B4%80%EB%A0%A8-%EC%9D%B4%EB%A1%A0-%EC%A0%95%EB%A6%AC/)
## Word2Vec
- 단어를 벡터화 하는 방법 중의 하나
- NNLM => RNNLM => CBOW & Skip-gram
- 단어의 벡터화를 학습하는 과정에서 가장 중요한 가정
    - [Distributional Hpyothesis](https://en.wikipedia.org/wiki/Distributional_semantics#Distributional_Hypothesis)
        - 언어 데이터의 대형 샘플에서 분포 속성을 기반으로 언어 항목 간의 의미론적 유사성을 정량화하고 분류하는 이론 및 방법
        - 동일한 맥락에서 사용되는 단어가 비슷한 의미를 갖는 경향이 있다는 의미론적 이론으로부터 유도됨

### 1)NNLM 
- 단어들을 N개의 one-hot encoding으로 벡터화하고
- 사전의 크기를 V, Projection Layer의 크기를 P라 할 때
- 각각의 벡터들은 V * P크기의 Projection Matrix에 의해 다음 레이어로 넘어가게 됨
#### 즉,  V차원의 단어벡터들을 P차원으로 바꿔줌
- 그리고 그 뒤는 기존의 NN와 유사(hidden layer 거치고 output layer), 각 단어들이 나올 확률 계산해서 weight 최적화
![image](https://user-images.githubusercontent.com/28910538/54740380-91b75b80-4bfe-11e9-9673-f7f2b14df3e7.png)


In [1]:
# gensim 설치
# pip install gensim
from gensim.models.word2vec import Word2Vec